In [1]:
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pandas as pd
import glob
import fnmatch
import os
import SimpleITK as sitk

# ANNOTATIONS

The annotation file is a csv file that contains one finding per line. Each line holds the SeriesInstanceUID of the scan, the x, y, and z position of each finding in world coordinates; and the corresponding diameter in mm. The annotation file contains 1186 nodules.

The list of annotations that are not used as reference standard will be provided. Each line holds the SeriesInstanceUID of the scan, the x, y, and z position of each finding in world coordinates; and the corresponding diameter in mm. It has to be noted that findings that were annotated as nodule < 3 mm and non-nodule have no diameter measurement.

In [2]:
annotations = pd.read_csv("/home/msmith/luna16/CSVFILES/annotations.csv")
annotations.head()
print(annotations.shape)

(1186, 5)


In [42]:
# Some Functions
def getPathSeriesuid(seriesuid):
    pattern = '*'+seriesuid+'*'
    for roots, dirs, files in os.walk('/home/msmith/luna16/'):
            for filename in fnmatch.filter(files,pattern):
                returnPath = os.path.join(roots,filename)
                #print(returnPath)
    return returnPath

def getAnnotationsFullPathCSV():
    # Includes full path of files in final column
    annotations["serisuidComplete"] = annotations["seriesuid"].apply(getPathSeriesuid) # Get complete paths of files (takes a while)
    annotations.to_csv("/CSVFILES/annotationsFullPaths.csv",index=0)

def load_itk_image(filename):
    itkimage = sitk.ReadImage(filename)
    numpyImage = sitk.GetArrayFromImage(itkimage)
    numpyOrigin = np.array(list(reversed(itkimage.GetOrigin())))
    numpySpacing = np.array(list(reversed(itkimage.GetSpacing())))
    return numpyImage, numpyOrigin, numpySpacing

'''
def load_itk_image(filename):
    itkimage = sitk.ReadImage(filename)
    numpyImage = sitk.GetArrayFromImage(itkimage)
    numpyOrigin = np.array(list(itkimage.GetOrigin()))
    numpySpacing = np.array(list(itkimage.GetSpacing()))
    return numpyImage, numpyOrigin, numpySpacing
'''

def worldToVoxelCoord(worldCoord, origin, spacing):
    stretchedVoxelCoord = np.absolute(worldCoord - origin)
    voxelCoord = stretchedVoxelCoord / spacing
    return voxelCoord

In [64]:
ofInterest = annotations[annotations["seriesuid"].str.contains("1.3.6.1.4.1.14519.5.2.1.6279.6001.134996872583497382954024478441")]

In [9]:
# Get random annotation from list
#randInt = np.random.randint(annotations.shape[0])
randInt = 130
randAnnotation = annotations.iloc[randInt]

noduleCoords = randAnnotation['coordX'],randAnnotation['coordY'],randAnnotation['coordZ']
pattern = '*'+randAnnotation['seriesuid']+'*'

imgPath = getPathSeriesuid(pattern)
rawPath = imgPath[:-3] + 'raw'
img = np.fromfile(rawPath,dtype='int16')
with open(imgPath[:-3] + 'mhd',"r") as f:
    imgData = f.read()
    mhdPath = rawPath[:-3] + 'mhd'
    
img, imgOrigin, imgSpacing = load_itk_image(mhdPath)

def printInfo():
    print("Using observation", randInt)
    print(randAnnotation)
    print("=="*50)
    print("=="*50)
    print(imgData)
    annotations[randInt:randInt+1]
printInfo()

('Using observation', 130)
seriesuid      1.3.6.1.4.1.14519.5.2.1.6279.6001.134996872583...
coordX                                                  -70.3231
coordY                                                  56.20438
coordZ                                                 -82.47946
diameter_mm                                             7.888814
Name: 130, dtype: object
ObjectType = Image
NDims = 3
BinaryData = True
BinaryDataByteOrderMSB = False
CompressedData = False
TransformMatrix = 1 0 0 0 1 0 0 0 1
Offset = -238.10000600000001 -220 -273.25
CenterOfRotation = 0 0 0
AnatomicalOrientation = RAI
ElementSpacing = 0.859375 0.859375 1.25
DimSize = 512 512 197
ElementType = MET_SHORT
ElementDataFile = 1.3.6.1.4.1.14519.5.2.1.6279.6001.134996872583497382954024478441.raw



In [ ]:
# Nodule voxel coordinates (nvc)
nvc = worldToVoxelCoord(noduleCoords[::-1],imgOrigin,imgSpacing)
nvc = np.array(nvc.astype(int))

#Get a range in which to subset the data
margin = 50
nvcMinus,nvcPlus = (nvc - margin).astype(int), (nvc + margin).astype(int)
def printNoduleInfo():
    print("Image shape ==>" + "\n" + str(img.shape))
    print("Nodule coordinates @"+"\n" + str(nvc))
    print("NVC margins ==>" + "\n" + str(nvcMinus) + "\n" + str(nvcPlus))

In [ ]:
from matplotlib.patches import Rectangle
rectangle = plt.Rectangle((nvcMinus[2], nvcMinus[1]),margin*2 ,margin*2, facecolor = 'none', ec='r')
circle = plt.Circle((nvc[2],nvc[1]),10, facecolor = 'none', ec='r')
plt.gca().add_patch(rectangle)
plt.gca().add_patch(circle)
plt.imshow(img[nvc[0]],cmap=cm.gray)
plt.show()


In [ ]:
for i in xrange(nvcMinus[0],nvcPlus[0],10):
    if i == nvc[0]:
        print("Nodule located here")
    print(i)
    plt.imshow(img[i,nvcMinus[1]:nvcPlus[1],nvcMinus[2]:nvcPlus[2]],cmap=cm.gray)
    plt.show()

In [ ]:
voxels = img.flatten().shape[0]

In [ ]:
plt.hist(img.flatten())
plt.show()

In [ ]:
#img1 = np.clip(img[59,180:220,140:180],-1000,-600)
img1 = img[59,180:220,140:180]
plt.hist(img1.flatten())
plt.show()

In [ ]:
img2 = np.clip(img1,-1000,-800)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
x,y = np.arange(0,img1.shape[0],1),np.arange(0,img2.shape[0],1)

In [ ]:
x,y = np.meshgrid(x,y)

In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot_surface(x,y,img1)

In [ ]:

x = noduleVoxelCoords[0]
y = noduleVoxelCoords[1]
z = noduleVoxelCoords[2]
for a in range(55,66,1):
    plt.figure(figsize=(7,7))
    img1=img[a,180:220,140:180]
    plt.imshow(np.clip(img1,-1000,-500),cmap=cm.gray)
    plt.show()

In [ ]:
from scipy.interpolate import RegularGridInterpolator

In [ ]:
# Get a 3D image eg. subset of img
subsetSize = 20
img3 = img[:subsetSize,:subsetSize,:subsetSize]
x,y,z = np.arange(subsetSize),np.arange(subsetSize),np.arange(subsetSize)
interpolator = RegularGridInterpolator((x,y,z),img3,bounds_error=False,fill_value=img3.mean())

In [ ]:
interpolator

# CANDIDATES

The candidates file is a csv file that contains nodule candidate per line. Each line holds the scan name, the x, y, and z position of each candidate in world coordinates, and the corresponding class. The list of candidates is provided for participants who are following the ‘false positive reduction’ track. Tutorial on how to view lesions given the location of candidates will be available on the Forum page.

The candidate locations are computed using three existing candidate detection algorithms [1-3]. As lesions can be detected by multiple candidates, those that are located <= 5 mm are merged. Using this method, 1120 out of 1186 nodules are detected with 551,065 false positives. For convenience, the corresponding class label (0 for non-nodule and 1 for nodule) for each candidate is provided in the list. It has to be noted that there can be multiple candidates per nodule.

In [ ]:
candidates = pd.read_csv("/home/msmith/luna16/CSVFILES/candidates.csv")
candidatesClass = candidates['class']
candidatesClass.hist(bins=2)
propClass1 = candidatesClass[candidatesClass==1].sum()/float(candidatesClass.size)
plt.title("Candidates class count. Proportion class 1 = "+str(propClass1))
plt.show()